In [ ]:
CREATE TABLE IF NOT EXISTS DATA_SLV.DIM_VENUES (
    VENUE_ID NUMBER(38,0),
    ID_CITY NUMBER(38,0),
    ADDRESS_1 VARCHAR,
    VENUE_NAME VARCHAR,
    RATING NUMBER(38,0),
    RATING_COUNT NUMBER(38,0),
    NORMALISED_RATING NUMBER(38,0)
);

MERGE INTO DATA_SLV.DIM_VENUES AS d
USING (
  SELECT DISTINCT
    TRY_TO_NUMBER(v.venue_id)                              AS venue_id,         -- NK
    TRY_TO_NUMBER(c.id_city)                               AS id_city,          -- FK 
    NULLIF(TRIM(v.address_1), '')                          AS address_1,
    NULLIF(TRIM(v.venue_name), '')                         AS venue_name,

    TRY_TO_NUMBER(ROUND(v.rating,              0))         AS rating,
    TRY_TO_NUMBER(ROUND(v.rating_count,        0))         AS rating_count,
    TRY_TO_NUMBER(ROUND(v.normalised_rating,   0))         AS normalised_rating
  FROM MEETUP_DB.DATA_BRZ.VENUES AS v
  LEFT JOIN DATA_SLV.DIM_CITIES AS c
    ON  UPPER(TRIM(v.city))    = UPPER(TRIM(c.city))
    AND COALESCE(UPPER(TRIM(v.state))  , '') = COALESCE(UPPER(TRIM(c.state))  , '')
    AND COALESCE(UPPER(TRIM(v.country)), '') = COALESCE(UPPER(TRIM(c.country)), '')
  WHERE TRY_TO_NUMBER(v.venue_id) IS NOT NULL
) AS s
ON d.VENUE_ID = s.venue_id

WHEN MATCHED THEN UPDATE SET
  d.ID_CITY            = s.id_city,
  d.ADDRESS_1          = s.address_1,
  d.VENUE_NAME         = s.venue_name,
  d.RATING             = s.rating,
  d.RATING_COUNT       = s.rating_count,
  d.NORMALISED_RATING  = s.normalised_rating

WHEN NOT MATCHED THEN INSERT (
  VENUE_ID, ID_CITY, ADDRESS_1, VENUE_NAME, RATING, RATING_COUNT, NORMALISED_RATING
) VALUES (
  s.venue_id, s.id_city, s.address_1, s.venue_name, s.rating, s.rating_count, s.normalised_rating
);